In [1]:
import os
path = os.path.join('/home/santiago/Documents/dev/reservoirpy')
import sys
sys.path.insert(0,path)
from reservoirpy.pvtpy import black_oil as bl
from reservoirpy.wellproductivitypy import pi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
ubh = pi.ubh(brand='longdi', nozzle=3.59, throat=6.09, depth=4300, injection_di=1.9, return_di=1.5)

In [4]:
issubclass(type(ubh),pi.als)

True

In [5]:
ubh.annulus_area()

0.02946034402972634

In [6]:
ubh.get_area('nozzle')

0.01568958098717009

In [7]:
ubh.get_area('throat')

0.04514992501689643

In [8]:
ubh.depth

array([4300, 4300, 4300, 4300, 4300, 4300, 4300, 4300, 4300, 4300, 4300,
       4300, 4300, 4300, 4300])

In [10]:
ubh.injection_di

array([1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9,
       1.9, 1.9])